# Synthetic Preference Generation

Generate synthetic preference pairs for DPO bootstrapping.

In [1]:
import sys
from pathlib import Path
import json
from collections import Counter

sys.path.append(str(Path('../core').resolve()))
from synthetic_preference_generator import (
    SyntheticPreferenceGenerator,
    IntentRule,
    PreferencePair,
    load_rules_from_generated_file
)

In [ ]:
generator = SyntheticPreferenceGenerator(
    confidence_threshold_high=0.7,
    confidence_threshold_low=0.4,
    min_novelty_score=0.3,
    require_platform_context=True,
    synthetic_weight=0.3
)

## Sample Rules

In [3]:
sample_rules = [
    IntentRule(
        rule_id='rule_1',
        description='User adjusts text size for readability',
        scope='artifact_property',
        abstraction_level='intermediate',
        triggering_actions=['modify_property'],
        confidence=0.85,
        platform_context={'platform': 'figma', 'extraction_method': 'plugin_api'},
        training_metadata={'suitable_for_training': True, 'novelty_score': 0.8, 'pattern_frequency': 'common'}
    ),
    IntentRule(
        rule_id='rule_2',
        description='User changes something',
        scope='structural',
        abstraction_level='general',
        triggering_actions=['modify'],
        confidence=0.35,
        platform_context={'platform': 'figma'},
        training_metadata={'suitable_for_training': False, 'novelty_score': 0.2, 'pattern_frequency': 'rare'}
    ),
    IntentRule(
        rule_id='rule_3',
        description='User modifies font size property of text element',
        scope='artifact_property',
        abstraction_level='specific',
        triggering_actions=['modify_property', 'text_font_size'],
        confidence=0.75,
        platform_context={'platform': 'figma', 'extraction_method': 'plugin_api'},
        training_metadata={'suitable_for_training': True, 'novelty_score': 0.5, 'pattern_frequency': 'common'}
    )
]

print(f"Sample rules: {len(sample_rules)}")
for rule in sample_rules:
    print(f"  {rule.rule_id}: {rule.description[:50]}... (conf: {rule.confidence:.2f})")

Sample rules: 3
  rule_1: User adjusts text size for readability... (conf: 0.85)
  rule_2: User changes something... (conf: 0.35)
  rule_3: User modifies font size property of text element... (conf: 0.75)


## Generate Preferences

In [4]:
trace_context = {'trace_id': 'trace_001', 'platform': 'figma'}

pairs = generator.generate_preferences(
    sample_rules,
    trace_context=trace_context,
    strategies=['confidence', 'quality', 'completeness', 'novelty', 'frequency']
)

print(f"Generated {len(pairs)} preference pairs")
print(f"\nBreakdown by source:")
for source, count in Counter(p.source for p in pairs).items():
    print(f"  {source}: {count}")

Generated 0 preference pairs

Breakdown by source:


In [5]:
print("Example Preference Pairs:\n")
for i, pair in enumerate(pairs[:3]):
    print(f"Pair {i+1} ({pair.source}, weight: {pair.weight}):")
    print(f"  Preferred: {pair.preferred.description[:60]}... (conf: {pair.preferred.confidence:.2f})")
    print(f"  Rejected:  {pair.rejected.description[:60]}... (conf: {pair.rejected.confidence:.2f})")
    print()

Example Preference Pairs:



## Format for DPO Training

In [ ]:
dpo_examples = generator.format_for_dpo(pairs, include_weights=True)

if dpo_examples:
    print("DPO Training Format (first example):")
    print(json.dumps(dpo_examples[0], indent=2))
else:
    print("No DPO examples generated (no preference pairs available)")

DPO Training Format (first example):


IndexError: list index out of range

## Quality Analysis

In [ ]:
if pairs:
    confidence_gaps = [p.preferred.confidence - p.rejected.confidence for p in pairs]
    print("Preference Quality Analysis:")
    print(f"  Total pairs: {len(pairs)}")
    print(f"  Mean confidence gap: {sum(confidence_gaps)/len(confidence_gaps):.3f}")
    print(f"  Min gap: {min(confidence_gaps):.3f}")
    print(f"  Max gap: {max(confidence_gaps):.3f}")

## Next Steps

- 03_rule_generation.ipynb: Generate rules from snapshots
- 04_dataset_building.ipynb: Build full training dataset